In [61]:
import plotly.graph_objects as go
import numpy as np
import geopandas as gpd
from scipy.spatial import Delaunay


# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file('ne_110m_admin_1_states_provinces.shp')

# Filter out only the Florida state
florida_gdf = gdf[gdf['name'] == 'Florida']

# Extract the boundary coordinates of Florida
florida_boundary = florida_gdf.geometry.values[0].exterior.coords.xy
florida_x = list(florida_boundary[0])
florida_y = list(florida_boundary[1])

# Coordinates and profits from before
store_locations = [
    (27.9944024, -81.7602544),
    (25.5516034, -80.6326916),
    (30.6953657, -88.0398913),
    (29.6516344, -82.3248262),
    (27.9944024, -81.7602544),
    (30.421309, -87.2169149),
    (28.6122187, -80.8075537),
    (26.9342246, -80.0942087),
    (29.1871986, -82.1400923),
    (30.6953657, -88.0398913)
]
store_profits = [500, 650, 800, 450, 700, 550, 650, 600, 720, 675]

# Extract x, y, z data for stores
x = [location[1] for location in store_locations]
y = [location[0] for location in store_locations]
z = store_profits

# Create figure
fig = go.Figure()

# Add 3D scatter plot for store data with color based on profit
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', 
                           marker=dict(size=5, color=z, colorscale='Viridis', colorbar=dict(title='Profit'))))

# Add pillars (lines) for each store point down to the x-y plane
for i in range(len(x)):
    fig.add_trace(go.Scatter3d(x=[x[i], x[i]], y=[y[i], y[i]], z=[z[i], 0], mode='lines', line=dict(color='black')))

# Calculate the average profits for the area around each boundary point of Florida
florida_avg_profits = []
for fx, fy in zip(florida_x, florida_y):
    distances = np.sqrt((np.array(x) - fx)**2 + (np.array(y) - fy)**2)
    nearby_profits = [profit for d, profit in zip(distances, z) if d < 1]  # Considering profits of stores within 1-degree distance
    florida_avg_profits.append(np.mean(nearby_profits) if nearby_profits else 0)  # Setting to 0 if no nearby stores

# Add Florida boundary with color based on average profits
fig.add_trace(go.Scatter3d(x=florida_x, y=florida_y, z=[0]*len(florida_x), mode='lines+markers', 
                           marker=dict(size=3, color=florida_avg_profits, colorscale='Viridis'),
                           line=dict(color='black')))

# Set axes properties
fig.update_layout(scene=dict(
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    zaxis_title='Profit ($ in thousands)')
)

# Display the plot
fig.show()


In [ ]:
# Coordinates and profits from before
store_locations = [
    (27.9944024, -81.7602544),
    (25.5516034, -80.6326916),
    (30.6953657, -88.0398913),
    (29.6516344, -82.3248262),
    (27.9944024, -81.7602544),
    (30.421309, -87.2169149),
    (28.6122187, -80.8075537),
    (26.9342246, -80.0942087),
    (29.1871986, -82.1400923),
    (30.6953657, -88.0398913)
]

In [68]:
import plotly.graph_objects as go
import numpy as np
import geopandas as gpd
from scipy.spatial import Delaunay

# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file('ne_110m_admin_1_states_provinces.shp')

# Filter out only the Florida state
florida_gdf = gdf[gdf['name'] == 'Florida']

# Extract the boundary coordinates of Florida
florida_boundary = florida_gdf.geometry.values[0].exterior.coords.xy
florida_x = list(florida_boundary[0])
florida_y = list(florida_boundary[1])

# Coordinates and profits from before
# Coordinates and profits from before
store_locations = [
    (27.9944024, -81.7602544),
    (25.5516034, -80.6326916),
    (30.6953657, -88.0398913),
    (29.6516344, -82.3248262),
    (27.9944024, -81.7602544),
    (30.421309, -87.2169149),
    (28.6122187, -80.8075537),
    (26.9342246, -80.0942087),
    (29.1871986, -82.1400923),
    (30.6953657, -88.0398913)
]

store_profits = [100, 100, 100, 100, 100, 100, 1000, 1000, 1000, 1000]

x = [location[1] for location in store_locations]
y = [location[0] for location in store_locations]
z = store_profits

# Triangulate the Florida boundary
tri = Delaunay(np.array([florida_x, florida_y]).T)
triangles = tri.simplices

# Calculate average profit for each triangle's center
triangle_avg_profits = []
for triangle in triangles:
    cx = np.mean([florida_x[i] for i in triangle])
    cy = np.mean([florida_y[i] for i in triangle])
    distances = np.sqrt((np.array(x) - cx)**2 + (np.array(y) - cy)**2)
    nearby_profits = [profit for d, profit in zip(distances, z) if d < 1]
    triangle_avg_profits.append(np.mean(nearby_profits) if nearby_profits else 0)

# Create figure
fig = go.Figure()

# Add 3D scatter plot for store data with color based on profit
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', 
                           marker=dict(size=5, color=z, colorscale='icefire', colorbar=dict(title='Profit'))))

# Add pillars (lines) for each store point down to the x-y plane
for i in range(len(x)):
    fig.add_trace(go.Scatter3d(x=[x[i], x[i]], y=[y[i], y[i]], z=[z[i], 0], mode='lines', line=dict(color='black')))

# Add Florida mesh with color based on average profits
fig.add_trace(go.Mesh3d(x=florida_x, y=florida_y, z=[0]*len(florida_x), i=triangles[:, 0], j=triangles[:, 1], k=triangles[:, 2], 
                        intensity=triangle_avg_profits, colorscale='icefire', cmin=min(z), cmax=max(z)))

# Set axes properties
fig.update_layout(scene=dict(
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    zaxis_title='Profit ($ in thousands)')
)

# Display the plot
fig.show()
